In [1]:
!pip install -r /requirements.txt
#from IPython import display
#display.clear_output()

## Retrieval

First we have to load the corpus.

In [2]:
from datasets import load_dataset
meta_corpus = load_dataset(
    "json",
    data_files="/data/corpus_chunks.jsonl",
    split="train"
).to_list()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
meta_corpus[19242]

{'title': 'Khoa học Trái Đất',
 'passage': 'Title: Khoa học Trái Đất\n\ncũng bao gồm những nghiên cứu về Trái Đất và các hành tinh lân cận khác trong không gian. Một số nhà khoa học Trái Đất sử dụng tri thức của họ về các hành tinh để định vị và phát triển tài nguyên năng lượng và khoáng sản. Số khác lại nghiên cứu tác động từ hoạt động của con người đến môi trường Trái Đất; từ đó thiết kế những biện pháp bảo vệ hành tinh. Ngoài ra, số còn lại thì đi sâu hơn về nghiên cứu các hiện tượng Trái Đất như núi lửa, động đất, và bão để giúp con người tránh những thảm hoạ tàn khốc của thiên nhiên. Khoa học Trái Đất có thể bao gồm các nghiên cứu về địa chất, thạch quyển, và cấu trúc quy mô lớn sâu bên trong lõi Trái Đất, cũng như là bầu khí quyển của',
 'id': 19242,
 'len': 155}

In [4]:
import copy
import unicodedata as ud
import re
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from tqdm.notebook import tqdm
import string

def split_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.lower().split()
    words = [word for word in words if len(word.strip()) > 0]
    return words

## initiate BM25 retriever
tokenized_corpus = [split_text(doc["passage"]) for doc in tqdm(meta_corpus)]
bm25 = BM25Okapi(tokenized_corpus)

  0%|          | 0/48530 [00:00<?, ?it/s]

In [5]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import pickle
from pyvi.ViTokenizer import tokenize
import numpy as np
from tqdm.notebook import tqdm
import torch

## initiate semantic rertiever
with open('/data/corpus_embedding_w150.pkl', 'rb') as f:
    corpus_embs = pickle.load(f)
device = "cuda"
embedder = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder').to(device)

2024-09-02 16:21:52.997468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 16:21:52.997570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 16:21:53.135409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

To improve from simple single BM25 retriever, we now incorporate a sentence transformer model to do the semantic search. \
We will use `bkai-foundation-models/vietnamese-bi-encoder` model which supports Vietnamese pretty well.

In [6]:
from copy import deepcopy
def retrieve(question, topk=50):
    """
    Get most relevant chunks to the question using combination of BM25 and semantic scores.
    """
    ## initialize query for each retriever (BM25 and semantic)
    tokenized_query = split_text(question)
    segmented_question = tokenize(question)
    question_emb = embedder.encode([segmented_question])
    question_emb /= np.linalg.norm(question_emb, axis=1)[:, np.newaxis]

    ## get BM25 and semantic scores
    bm25_scores = bm25.get_scores(tokenized_query)
    semantic_scores = question_emb @ corpus_embs.T
    semantic_scores = semantic_scores[0]

    ## update chunks' scores.
    max_bm25_score = max(bm25_scores)
    min_bm25_score = min(bm25_scores)
    def normalize(x):
        return (x - min_bm25_score + 0.1) / \
        (max_bm25_score - min_bm25_score + 0.1)

    corpus_size = len(meta_corpus)
    for i in range(corpus_size):
        meta_corpus[i]["bm25_score"] = bm25_scores[i]
        meta_corpus[i]["bm25_normed_score"] = normalize(bm25_scores[i])
        meta_corpus[i]["semantic_score"] = semantic_scores[i]

    ## compute combined score (BM25 + semantic)
    for passage in meta_corpus:
        passage["combined_score"] = passage["bm25_normed_score"] * 0.4 + \
                                    passage["semantic_score"] * 0.6

    ## sort passages by the combined score
    sorted_passages = sorted(meta_corpus, key=lambda x: x["combined_score"], reverse=True)
    return sorted_passages[:topk]

## Smoothing contexts

Using combination of BM25 and semantic score may still not yield the best result (because of the reasons you will see at the end of this section) \
We can do better by applying several techniques to form suitable contexts to feed to the LLM. \
The following blocs will implement some utility functions with such techniques.

In [7]:
from copy import deepcopy
from underthesea import sent_tokenize
def extract_consecutive_subarray(numbers):
    subarrays = []
    current_subarray = []
    for num in numbers:
        if not current_subarray or num == current_subarray[-1] + 1:
            current_subarray.append(num)
        else:
            subarrays.append(current_subarray)
            current_subarray = [num]

    subarrays.append(current_subarray)  # Append the last subarray
    return subarrays

def merge_contexts(passages):
    passages_sorted_by_id = sorted(passages, key=lambda x: x["id"])

    psg_ids = [x["id"] for x in passages_sorted_by_id]
    consecutive_ids = extract_consecutive_subarray(psg_ids)

    merged_contexts = []
    b = 0
    for ids in consecutive_ids:
        psgs = passages_sorted_by_id[b:b+len(ids)]
        
        # Group passages by title within the consecutive IDs
        title_groups = {}
        for psg in psgs:
            title = psg["title"]
            if title not in title_groups:
                title_groups[title] = []
            title_groups[title].append(psg)
        
        # Merge passages in each title group
        for title, group in title_groups.items():
            if len(group) == 1:
                # If only one passage in the group, add it as is
                psg = group[0]
                merged_contexts.append(dict(
                    title=psg['title'],
                    passage=psg['passage'],
                    score=psg["combined_score"],
                    merged_from_ids=[psg["id"]]
                ))
            else:
                # Merge passages with the same 
                psg_texts = [clean_passage(x) for x in group]
                merged = f"Title: {group[0]['title']}\n\n" + " ".join(psg_texts)
                merged_contexts.append(dict(
                    title=group[0]['title'],
                    passage=merged,
                    score=max([x["combined_score"] for x in group]),
                    merged_from_ids=[x["id"] for x in group]
                ))

        b = b + len(ids)

    return merged_contexts



def discard_contexts(passages):
    sorted_passages = sorted(passages, key=lambda x: x["score"])
    if len(sorted_passages) == 1:
        return sorted_passages
    else:
        shortened = deepcopy(sorted_passages)
        for i in range(len(sorted_passages) - 1):
            current, next = sorted_passages[i], sorted_passages[i+1]
            if next["score"] - current["score"] >= 0.05:
                shortened = sorted_passages[i+1:]
        return shortened

def expand_context(passage, word_window=60, n_sent=3):
    # psg_id = passage["id"]
    merged_from_ids = passage["merged_from_ids"]
    title = passage["title"]
    prev_id = merged_from_ids[0] - 1
    next_id = merged_from_ids[-1] + 1

    texts = []
    if prev_id in range(0, len(meta_corpus)):
        prev_psg = meta_corpus[prev_id]
        if prev_psg["title"] == title:
            prev_text = clean_passage(prev_psg)
            # prev_text = " ".join(prev_text.split()[-word_window:])
            prev_text = " ".join(sent_tokenize(prev_text)[-n_sent:])
            texts.append(prev_text)

    texts.append(clean_passage(passage))

    if next_id in range(0, len(meta_corpus)):
        next_psg = meta_corpus[next_id]
        if next_psg["title"] == title:
            next_text = clean_passage(next_psg)
            # next_text = " ".join(next_text.split()[:word_window])
            next_text = " ".join(sent_tokenize(next_text)[:n_sent])
            texts.append(next_text)

    expanded_text = " ".join(texts)
    expanded_text = f"Title: {title}\n{expanded_text}"
    new_passage = deepcopy(passage)
    new_passage["passage"] = expanded_text
    return new_passage

def expand_contexts(passages, word_window=60, n_sent=3):
    new_passages = [expand_context(passage) for passage in passages]
    return new_passages

def collapse(passages):
    new_passages = deepcopy(passages)
    titles = {}
    for passage in new_passages:
        title = passage["title"]
        if not titles.get(title):
            titles[title] = [passage]
        else:
            titles[title].append(passage)
    best_passages = []
    for k, v in titles.items():
        best_passage = max(v, key= lambda x: x["score"])
        best_passages.append(best_passage)
    sorted_best_passages = sorted(best_passages, key=lambda x: x["score"], reverse=True)
    return sorted_best_passages

# def truncate_title(text):
#     newline_pos = text.find('\n')
#     if "Title:" in text and newline_pos != -1:
#         truncated_text = text[newline_pos + 1:]
#         return truncated_text
#     else:
#         return text

def final_clean(passages):
    output = ""
    for i in passages:
        output += clean_passage(i)
    return output

def clean_passage(entry):
    title = entry["title"]
    passage = entry["passage"]
    cleaned_passage = passage[(len(title)+8):].strip()
    # Define the pattern to match "Title: title"
#     pattern = re.compile(r"^Title:\s*" + re.escape(title) + r"\s*\n\n", re.IGNORECASE)
    
    # Substitute the matched pattern with an empty string
#     cleaned_passage = re.sub(pattern, "", passage)
#     cleaned_passage = cleaned_passage.replace('\n', '')
    return cleaned_passage
        

Note that, with our current chunking strategy, each chunk is a passage of exact 150 words (separated by space), not a comprehensive paragraph. The following function will transform retrieved chunks into whole paragraphs. This function also does some heuristics to expand the context window and discard seem-to-be irrelevant contexts.

In [8]:
def smooth_contexts(passages):
    """Make the context fed to the LLM better.
    Args:
        passages (list): Chunks retrieved from BM25 + semantic retrieval.

    Returns:
        list: List of whole paragraphs, usually will be more relevant to the initital question.
    """
    # 1. If consecutive chunks are retrieved, merge them into one big chunk to ensure the continuity.
    merged_contexts = merge_contexts(passages)
    # 2. A heuristic to discard irrelevevant contexts.
    # It seems to be better to only keep what are elevant so that the model can focus.
    # Also this reduce #tokens LLM has to read.
    shortlisted_contexts = discard_contexts(merged_contexts)
    # 3. Another heuristic. this step is to take advantage of long context understanding of the LLM.
    # In many cases, the retrieved passages are just consecutive words, not a comprehensive paragraph.
    # This is to expand the passage to the whole paragraph that surrounds it.
    # My intuition about this is that whole paragraph will add necessary and relevant information.
    expanded_contexts = expand_contexts(shortlisted_contexts)
    # 4. Now after all the merging and expanding, if what are left for us is more than one paragraphs
    # from the same wiki page, then we will only take paragraph with highest retrieval score.
    collapsed_contexts = collapse(expanded_contexts)
    final_clean_contexts = final_clean(collapsed_contexts)
    return final_clean_contexts

In [9]:
def get_prompts(question, topk=3):
    top_passages = retrieve(question, topk=topk)
    contexts = smooth_contexts(top_passages)
    xddd = {'question': question,'context': contexts}    
    return xddd

In [10]:
## I encourage you to investigate each smoothing step using the following example
## to understand the benefit of them.
## You will see that after each step, we will obtain "better" contexts.

question = "Phạm Đức Thành là ai?"
top_passages = retrieve(question, topk=10)
merged_contexts = merge_contexts(top_passages)
shortlisted_contexts = discard_contexts(merged_contexts)
expanded_contexts = expand_contexts(shortlisted_contexts)
collapsed_contexts = collapse(expanded_contexts)
final_clean_contexts = final_clean(collapsed_contexts)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
## Uncomment each of these variable to see the differences.

# top_passages
# merged_contexts
# shortlisted_contexts
# expanded_contexts
collapsed_contexts

[{'title': 'Phạm Đức Hải',
  'passage': 'Title: Phạm Đức Hải\nPhạm Đức Hải (sinh năm 1963) là một chính khách Việt Nam. Ông hiện là Phó Trưởng ban Tuyên giáo Thành ủy, Phó Trưởng ban chuyên trách Ban Chỉ đạo phòng, chống dịch Covid-19 và phục vụ hồi kinh tế TP.HCM. Ông từng là Thành ủy viên, Phó Chủ tịch Hội đồng nhân dân Thành phố Hồ Chí Minh, nhiệm kỳ 2016 - 2021 Lý lịch và học vấn. Phạm Đức Hải sinh ngày 29 tháng 7 năm 1963, quê quán xã Nghĩa Phú, huyện Nghĩa Hưng, tỉnh Nam Định. Ông được kết nạp vào Đảng Cộng sản Việt Nam ngày 17 tháng 9 năm 1994. Trình độ học vấn: Thạc sĩ Kinh tế, Cử nhân Chính trị. Sự nghiệp. Từ 1983 - 1989: Giáo viên kiêm Tổng Phụ trách Đội trường Hiệp Phước 2, huyện Nhà Bè; sau đó là cán bộ phụ trách Đội Thiếu niên Tiền phong Hồ Chí Minh thuộc Phòng Giáo dục và Đào tạo huyện Nhà Bè. Từ 1989 - 1996: Cán bộ chuyên trách của Đoàn Thanh niên Cộng sản Hồ Chí Minh thuộc Thành phố Hồ Chí Minh, lần lượt giữ các chức vụ Phó Trưởng ban, Trưởng ban Thiếu nhi Trường học, C

## Generate

In [12]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
model_path = "vinai/PhoGPT-4B-Chat"  

config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(model_path, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True)
model.to("cuda")
model.eval()  

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)  

PROMPT_TEMPLATE = "### Câu hỏi: {instruction}\n### Trả lời:"  

# Some instruction examples
# instruction = "Viết bài văn nghị luận xã hội về {topic}"
# instruction = "Viết bản mô tả công việc cho vị trí {job_title}"
# instruction = "Sửa lỗi chính tả:\n{sentence_or_paragraph}"
# instruction = "Dựa vào văn bản sau đây:\n{text}\nHãy trả lời câu hỏi: {question}"
# instruction = "Tóm tắt văn bản:\n{text}"
# instruction = "Sửa lỗi chính tả:\nTriệt phá băng nhóm kướp ô tô, sử dụng \"vũ khí nóng\""

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

norm.py:   0%|          | 0.00/3.12k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


blocks.py:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [13]:
def get_prompts(question, topk=3):
    top_passages = retrieve(question, topk=topk)
    contexts = smooth_contexts(top_passages)
    instruction = "Dựa vào văn bản sau đây:\n{text}\nHãy trả lời câu hỏi: {question}".format_map({"text": contexts, "question": question})
    input_prompt = PROMPT_TEMPLATE.format_map({"instruction": instruction})   
    return input_prompt

In [14]:
def rag(question, topk=3):
    prompt = get_prompts(question, topk=topk)
    input_ids = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(  
    inputs=input_ids["input_ids"].to("cuda"),  
    attention_mask=input_ids["attention_mask"].to("cuda"),  
    do_sample=True,  
    temperature=1.0,  
    top_k=50,  
    top_p=0.9,  
    max_new_tokens=1024,  
    eos_token_id=tokenizer.eos_token_id,  
    pad_token_id=tokenizer.pad_token_id)  

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
    response = response.split("### Trả lời:")[1]
    print(response)

In [15]:
questions = [
    "Chiến tranh Việt Nam bắt đầu vào khi nào?",
    "Chiến tranh Việt Nam kết thúc vào khi nào?",
    "Chiến tranh Việt Nam kéo dài trong bao lâu?"
    
]
for question in questions:
    print(f"Question: {question}")
    output = rag(question)
    print("---" * 30)

Question: Chiến tranh Việt Nam bắt đầu vào khi nào?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


Chiến tranh Việt Nam bắt đầu vào ngày 1 tháng 11 năm 1955.
------------------------------------------------------------------------------------------
Question: Chiến tranh Việt Nam kết thúc vào khi nào?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chiến tranh Việt Nam kết thúc vào ngày 30 tháng 4 năm 1975, giải phóng miền Nam thống nhất đất nước.
------------------------------------------------------------------------------------------
Question: Chiến tranh Việt Nam kéo dài trong bao lâu?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chiến tranh Việt Nam kéo dài từ năm 1954 đến năm 1975.
------------------------------------------------------------------------------------------


In [ ]:
while True:
    question = input(f"Câu hỏi:")
    output = rag(question)
    print("---" * 30)

Câu hỏi: Thành phố nào có kinh tế phát triển nhất Việt Nam?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Thành phố Hồ Chí Minh là thành phố có kinh tế phát triển nhất Việt Nam.
------------------------------------------------------------------------------------------


Câu hỏi: Đội Thiếu niên Tiền phong Hồ Chí Minh được thành lập vào ngày nào?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Đội Thiếu niên Tiền phong Hồ Chí Minh được thành lập vào ngày 15 tháng 5 năm 1941.
------------------------------------------------------------------------------------------


Câu hỏi: Ngày Quốc khánh 2/9 là ngày gì ở Việt Nam?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Ngày Quốc khánh 2/9 là ngày Quốc khánh ở Việt Nam.
------------------------------------------------------------------------------------------
